In [1]:
import os
import time
import math
import csv
import random
from pathlib import Path
from datetime import datetime

import psutil
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from tokenizers import ByteLevelBPETokenizer
from transformers import GPT2TokenizerFast
from datasets import load_dataset

In [13]:
# -------------------- USER CONFIG --------------------
OUT_DIR = Path("/kaggle/working/hindi-gpt")
CKPT_DIR = OUT_DIR / "checkpoints"
TK_DIR = OUT_DIR / "tokenizer"
LOG_DIR = OUT_DIR / "logs"
OUT_DIR.mkdir(parents=True, exist_ok=True)
CKPT_DIR.mkdir(exist_ok=True)
TK_DIR.mkdir(exist_ok=True)
LOG_DIR.mkdir(exist_ok=True)

In [51]:
BLOCK_SIZE = 128    

In [2]:
# Model / tokenizer config
TEACHER_BLOCK_SIZE = 128
TEACHER_D_MODEL = 512
TEACHER_N_HEAD = 8
TEACHER_N_LAYER = 8
TEACHER_DROPOUT = 0.1
TEACHER_VOCAB_SIZE = 32000


In [39]:
# Model / tokenizer config (student)
STUDENT_BLOCK_SIZE = 128
STUDENT_D_MODEL = 256
STUDENT_N_HEAD = 4
STUDENT_N_LAYER = 4
STUDENT_DROPOUT = 0.1
STUDENT_VOCAB_SIZE = 32000


In [40]:

# Training config
TOTAL_STEPS = 50000   
BATCH_SIZE = 16       
EVAL_INTERVAL = 2000
EVAL_ITERS = 100
LEARNING_RATE = 3e-4
SAVE_EVERY = 2000
SEED = 1337
# Data selection
TARGET_BYTES = 1 * 1024**3   # ~1 GB raw text
MAX_DOCS = None  

In [10]:
# Misc
USE_GRAD_ACCUM = False
GRAD_ACCUM_STEPS = 2
GENERATE_TOKENS = 200
INFERENCE_PROMPT = "भारत की तकनीकी प्रगति के बारे में विस्तृत जानकारी दीजिए और बताइए कि आने वाले वर्षों में भारत किस दिशा में आगे बढ़ेगा।"

In [11]:
# reproducibility
random.seed(SEED)
torch.manual_seed(SEED)

# device info
device = "cuda" if torch.cuda.is_available() else "cpu"
n_gpus = torch.cuda.device_count()
print(f"Device: {device}, GPUs: {n_gpus}")
if device == "cuda":
    for i in range(n_gpus):
        print(" GPU", i, ":", torch.cuda.get_device_name(i))
print("RAM (GB):", psutil.virtual_memory().total / 1e9)

Device: cuda, GPUs: 1
 GPU 0 : Tesla P100-PCIE-16GB
RAM (GB): 33.662472192


In [12]:
# -------------------- LOAD DATASET (nisram) & extract ~1GB --------------------
print("Loading nis12ram/nisram-hindi-text-0.0 ...")
ds = load_dataset("nis12ram/nisram-hindi-text-0.0", split="train")

all_texts = ds["text"]
print("Total rows in dataset:", len(all_texts))

indices = list(range(len(all_texts)))
random.shuffle(indices)

texts = []
acc_bytes = 0
count = 0
for i in indices:
    if MAX_DOCS is not None and count >= MAX_DOCS:
        break
    t = all_texts[i]
    if not t:
        continue
    b = t.encode("utf-8")
    acc_bytes += len(b)
    texts.append(t)
    count += 1
    if count % 20000 == 0:
        print(f"Collected {count:,} docs, {acc_bytes/1e9:.3f} GB")
    if acc_bytes >= TARGET_BYTES:
        break

print(f"Selected {len(texts):,} documents, approx {acc_bytes/1e9:.3f} GB")




Loading nis12ram/nisram-hindi-text-0.0 ...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00006.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

data/train-00001-of-00006.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

data/train-00002-of-00006.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

...

data/train-00004-of-00006.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

data/train-00005-of-00006.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/601628 [00:00<?, ? examples/s]

Total rows in dataset: 601628
Collected 20,000 docs, 0.091 GB
Collected 40,000 docs, 0.181 GB
Collected 60,000 docs, 0.271 GB
Collected 80,000 docs, 0.361 GB
Collected 100,000 docs, 0.451 GB
Collected 120,000 docs, 0.542 GB
Collected 140,000 docs, 0.632 GB
Collected 160,000 docs, 0.722 GB
Collected 180,000 docs, 0.811 GB
Collected 200,000 docs, 0.902 GB
Collected 220,000 docs, 0.991 GB
Selected 238,301 documents, approx 1.074 GB


NameError: name 'OUT_DIR' is not defined

In [14]:
raw_text_file = OUT_DIR / "nisram_hi_raw.txt"
with open(raw_text_file, "w", encoding="utf-8") as f:
    for t in texts:
        line = t.replace("\r\n", "\n").strip()
        if line == "":
            continue
        f.write(line + "\n")
print("Saved raw text file:", raw_text_file, "size GB:", raw_text_file.stat().st_size / 1e9)

Saved raw text file: /kaggle/working/hindi-gpt/nisram_hi_raw.txt size GB: 1.073857009


In [17]:
# -------------------- TRAIN BYTE-LEVEL BPE TOKENIZER --------------------
print("Training ByteLevel BPE tokenizer (vocab_size = {}) ...".format(TEACHER_VOCAB_SIZE))
bpe = ByteLevelBPETokenizer()
bpe.train(files=[str(raw_text_file)], vocab_size=TEACHER_VOCAB_SIZE, min_frequency=2,
          special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])
bpe.save_model(str(TK_DIR))
print("Tokenizer model files saved to:", TK_DIR)

# wrap as HuggingFace fast tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained(str(TK_DIR))
tokenizer.add_special_tokens({
    "bos_token": "<s>",
    "eos_token": "</s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
})
print("Tokenizer vocab size reported:", tokenizer.vocab_size)

# -------------------- TOKENIZE & BUILD 1D TOKEN STREAM --------------------
print("Tokenizing texts and building token stream ...")
all_ids = []
batch_tokenize = 500
for i in range(0, len(texts), batch_tokenize):
    batch = texts[i:i+batch_tokenize]
    enc = tokenizer(batch, add_special_tokens=False)
    for seq in enc["input_ids"]:
        all_ids.extend(seq + [tokenizer.eos_token_id])
    if (i // batch_tokenize) % 20 == 0:
        print(f"Tokenized up to doc {i+len(batch):,}, tokens so far: {len(all_ids):,}")

data = torch.tensor(all_ids, dtype=torch.long)
print("Total tokens in dataset:", len(data))

Training ByteLevel BPE tokenizer (vocab_size = 32000) ...



Tokenizer model files saved to: /kaggle/working/hindi-gpt/tokenizer
Tokenizer vocab size reported: 32000
Tokenizing texts and building token stream ...
Tokenized up to doc 500, tokens so far: 580,802
Tokenized up to doc 10,500, tokens so far: 11,988,665
Tokenized up to doc 20,500, tokens so far: 23,357,119
Tokenized up to doc 30,500, tokens so far: 34,779,078
Tokenized up to doc 40,500, tokens so far: 46,118,471
Tokenized up to doc 50,500, tokens so far: 57,335,192
Tokenized up to doc 60,500, tokens so far: 68,695,945
Tokenized up to doc 70,500, tokens so far: 80,066,706
Tokenized up to doc 80,500, tokens so far: 91,339,023
Tokenized up to doc 90,500, tokens so far: 102,694,022
Tokenized up to doc 100,500, tokens so far: 114,028,601
Tokenized up to doc 110,500, tokens so far: 125,344,266
Tokenized up to doc 120,500, tokens so far: 136,747,126
Tokenized up to doc 130,500, tokens so far: 148,175,479
Tokenized up to doc 140,500,

In [19]:
# -------------------- TRAIN / VAL SPLIT --------------------
n = int(0.9 * len(data))
train_data = data[:n].to(device)
val_data = data[n:].to(device)
print("Train tokens:", len(train_data), "Val tokens:", len(val_data))

# -------------------- BATCH / GET_BATCH --------------------
def get_batch(split):
    data_local = train_data if split == "train" else val_data
    max_start = len(data_local) - BLOCK_SIZE - 1
    ix = torch.randint(0, max_start, (BATCH_SIZE,))
    x = torch.stack([data_local[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data_local[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x, y

Train tokens: 242930408 Val tokens: 26992268


In [41]:
# -------------------- TEACHER MODEL DEFINITION --------------------
class TeacherHead(nn.Module):
    def __init__(self, d_model, head_size, block_size, dropout):
        super().__init__()
        self.key = nn.Linear(d_model, head_size, bias=False)
        self.query = nn.Linear(d_model, head_size, bias=False)
        self.value = nn.Linear(d_model, head_size, bias=False)

        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * (q.size(-1) ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        return wei @ v


class TeacherMultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head, head_size, dropout, block_size):
        super().__init__()
        self.heads = nn.ModuleList([
            TeacherHead(d_model, head_size, block_size, dropout)
            for _ in range(n_head)
        ])

        self.proj = nn.Linear(n_head * head_size, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        return self.dropout(self.proj(out))


class TeacherFeedForward(nn.Module):
    def __init__(self, d_model, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)


class TeacherBlock(nn.Module):
    def __init__(self, d_model, n_head, block_size, dropout):
        super().__init__()
        head_size = d_model // n_head

        self.sa = TeacherMultiHeadAttention(
            d_model, n_head, head_size, dropout, block_size
        )
        self.ffwd = TeacherFeedForward(d_model, dropout)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class TeacherGPT(nn.Module):
    def __init__(self,
                 vocab_size,
                 block_size,
                 d_model,
                 n_head,
                 n_layer,
                 dropout):

        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, d_model)
        self.position_embedding_table = nn.Embedding(block_size, d_model)

        self.blocks = nn.Sequential(
            *[TeacherBlock(d_model, n_head, block_size, dropout) for _ in range(n_layer)]
        )

        self.ln_f = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb

        x = self.blocks(x)
        x = self.ln_f(x)

        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(B*T, -1),
                targets.view(B*T),
                reduction="mean"
            )

        return logits, loss


In [53]:
teacher = TeacherGPT(
    vocab_size=32000,
    d_model=512,
    n_head=8,
    n_layer=8,
    block_size=128,
    dropout=0.1
)


checkpoint = torch.load(teacher_ckpt_path, map_location="cpu")
teacher.load_state_dict(checkpoint, strict=True)
teacher = teacher.to(device)

teacher.eval()
print("Teacher model loaded")
for p in teacher.parameters():
    p.requires_grad = False

teacher_params = sum(p.numel() for p in teacher.parameters())
print(f"✓ Teacher loaded: {teacher_params:,} parameters ({teacher_params/1e6:.2f}M)")


Teacher model loaded
✓ Teacher loaded: 58,073,344 parameters (58.07M)


In [43]:
# -------------------- STUDENT MODEL DEFINITION (GPT-style) --------------------
class StudentAFTSimpleHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.hidden_dim = head_size
        self.to_q = nn.Linear(STUDENT_D_MODEL, head_size, bias=False)
        self.to_k = nn.Linear(STUDENT_D_MODEL, head_size, bias=False)
        self.to_v = nn.Linear(STUDENT_D_MODEL, head_size, bias=False)
        self.project = nn.Linear(head_size, head_size)

    def forward(self, x):
        B, T, _ = x.shape
        Q = self.to_q(x)
        K = self.to_k(x)
        V = self.to_v(x)

        K_soft = F.softmax(K, dim=1)
        weights = (K_soft * V).sum(dim=1, keepdim=True)
        Q_sig = torch.sigmoid(Q)
        Yt = Q_sig * weights

        return self.project(Yt)

class StudentAFTSimpleMultiHead(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([StudentAFTSimpleHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, STUDENT_D_MODEL)
        self.dropout = nn.Dropout(STUDENT_DROPOUT)

    def forward(self, x):
        head_outs = [h(x) for h in self.heads]
        out = torch.cat(head_outs, dim=-1)
        out = self.proj(out)
        return self.dropout(out)

class StudentFeedForward(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model),
            nn.Dropout(STUDENT_DROPOUT),
        )
    def forward(self, x):
        return self.net(x)

class StudentBlock(nn.Module):
    def __init__(self, d_model, n_head):
        super().__init__()
        head_size = d_model // n_head
        self.sa = StudentAFTSimpleMultiHead(n_head, head_size)
        self.ffwd = StudentFeedForward(d_model)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class StudentGPT(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(STUDENT_VOCAB_SIZE, STUDENT_D_MODEL)
        self.position_embedding_table = nn.Embedding(STUDENT_BLOCK_SIZE, STUDENT_D_MODEL)

        self.blocks = nn.Sequential(
            *[StudentBlock(STUDENT_D_MODEL, STUDENT_N_HEAD) for _ in range(STUDENT_N_LAYER)]
        )

        self.ln_f = nn.LayerNorm(STUDENT_D_MODEL)
        self.lm_head = nn.Linear(STUDENT_D_MODEL, STUDENT_VOCAB_SIZE)

        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_indices = torch.arange(T, device=idx.device)
        pos_emb = self.position_embedding_table(pos_indices)
        x = tok_emb + pos_emb.unsqueeze(0)

        x = self.blocks(x)
        x = self.ln_f(x)

        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(B * T, STUDENT_VOCAB_SIZE),
                targets.view(B * T)
            )

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -STUDENT_BLOCK_SIZE:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / max(1e-8, temperature)
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_id), dim=1)
        return idx


In [44]:
model = StudentGPT()
model.to("cuda")       # or "cpu"

StudentGPT(
  (token_embedding_table): Embedding(32000, 256)
  (position_embedding_table): Embedding(128, 256)
  (blocks): Sequential(
    (0): StudentBlock(
      (sa): StudentAFTSimpleMultiHead(
        (heads): ModuleList(
          (0-3): 4 x StudentAFTSimpleHead(
            (to_q): Linear(in_features=256, out_features=64, bias=False)
            (to_k): Linear(in_features=256, out_features=64, bias=False)
            (to_v): Linear(in_features=256, out_features=64, bias=False)
            (project): Linear(in_features=64, out_features=64, bias=True)
          )
        )
        (proj): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): StudentFeedForward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=256, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      

In [46]:
student_params = sum(p.numel() for p in model.parameters())
compression_ratio = teacher_params / student_params
print(f"✓ Student created: {student_params:,} parameters ({student_params/1e6:.2f}M)")
print(f"📊 Compression ratio: {compression_ratio:.2f}x smaller")

✓ Student created: 19,671,808 parameters (19.67M)
📊 Compression ratio: 2.95x smaller


In [47]:
# KD hyperparameters
TEMPERATURE = 2.0
ALPHA = 0.5


In [48]:
def kd_loss(student_logits, teacher_logits, targets,
            T=TEMPERATURE, alpha=ALPHA):
    """
    KD loss = alpha * hard CE + (1-alpha) * soft KL.

    student_logits: (B, T, V)
    teacher_logits: (B, T, V)
    targets      : (B, T)
    """
    B, Tseq, V = student_logits.shape

    # Hard loss: CE over all tokens (mean per token)
    ce = F.cross_entropy(
        student_logits.view(B * Tseq, V),
        targets.view(B * Tseq)
    )

    # Soft loss: KL over all tokens (mean per token)
    student_logp = F.log_softmax(
        student_logits.view(B * Tseq, V) / T, dim=-1
    )
    teacher_soft = F.softmax(
        teacher_logits.view(B * Tseq, V) / T, dim=-1
    )

    kl = F.kl_div(
        student_logp,
        teacher_soft,
        reduction="batchmean",  # now truly mean over B*T tokens
    ) * (T * T)

    return alpha * ce + (1.0 - alpha) * kl

In [49]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scaler = torch.amp.GradScaler(enabled=(device=="cuda"))

In [57]:
# -------------------- EVAL / METRICS --------------------
@torch.no_grad()
def estimate_loss():
    was_training = model.training
    model.eval()
    out = {}
    for split in ["train", "val"]:
        losses = []
        iters = min(EVAL_ITERS,  max(1, (len(train_data) if split=="train" else len(val_data)) // (BATCH_SIZE * 2)))
        for _ in range(iters):
            xb, yb = get_batch(split)
            xb = xb.to(device); yb = yb.to(device)
            if device == "cuda":
                with torch.cuda.amp.autocast():
                    _, loss = model(xb, yb)
            else:
                _, loss = model(xb, yb)
            losses.append(loss.mean().item())
        out[split] = float(np.mean(losses))
    if was_training:
        model.train()
    return out

def ppl_from_loss(loss_val):
    return math.exp(loss_val) if loss_val < 50 else float("inf")

# -------------------- GENERATION (greedy helper) --------------------
@torch.no_grad()
def generate_text(prompt, max_new_tokens=200):
    model.eval()
    tokens = tokenizer.encode(prompt)
    idx = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)
    idx = model.generate(idx, max_new_tokens=max_new_tokens, temperature=1.0)
    text = tokenizer.decode(idx.squeeze().tolist(), skip_special_tokens=True)
    model.train()
    return text
# -------------------- LOGGING SETUP (ensure these exist) --------------------
log_csv = LOG_DIR / "train_log.csv"

# If starting fresh, create CSV header
if not log_csv.exists():
    with open(log_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["step","train_loss_iter","eval_train_loss","eval_val_loss","eval_train_ppl","eval_val_ppl","step_time_s","save_time_s","timestamp"])

# -------------------- RESET LOGGING LISTS --------------------
loss_history = []
eval_history = []
perp_history = []
timings = []

# -------------------- TRAIN LOOP WITH KD --------------------
print("Starting KD training for", TOTAL_STEPS, "steps ...")
global_start = time.time()
step = 0

try:
    init = estimate_loss()
    print(f"Initial eval - Train {init['train']:.4f}, Val {init['val']:.4f}")
except Exception as e:
    print("Initial eval failed:", e)
    init = {"train": None, "val": None}

model.train()   # student
teacher.eval()  # teacher stays frozen

while step < TOTAL_STEPS:
    step += 1
    t0 = time.time()

    xb, yb = get_batch("train")
    xb = xb.to(device)
    yb = yb.to(device)

    # Teacher forward
    with torch.no_grad():
        teacher_logits, _ = teacher(xb, None)

    # Student forward
    if device == "cuda":
        with torch.cuda.amp.autocast():
            student_logits, _ = model(xb, None)
            loss_raw = kd_loss(student_logits, teacher_logits, yb)
    else:
        student_logits, _ = model(xb, None)
        loss_raw = kd_loss(student_logits, teacher_logits, yb)

    loss = loss_raw.mean()
    loss_to_backprop = loss / (GRAD_ACCUM_STEPS if USE_GRAD_ACCUM else 1)

    # Backprop
    if device == "cuda":
        scaler.scale(loss_to_backprop).backward()
    else:
        loss_to_backprop.backward()

    # Optimizer step
    if (not USE_GRAD_ACCUM) or (step % GRAD_ACCUM_STEPS == 0):
        if device == "cuda":
            scaler.step(optimizer)
            scaler.update()
        else:
            optimizer.step()
        optimizer.zero_grad(set_to_none=True)

    step_time = time.time() - t0
    iter_train_loss = float(loss.item())
    loss_history.append((step, iter_train_loss))

    # Eval and checkpoint
    save_time_s = ""
    train_loss_eval = None
    val_loss_eval = None
    val_ppl = None

    if step % EVAL_INTERVAL == 0 or step == TOTAL_STEPS:
        t_eval_start = time.time()
        metrics = estimate_loss()
        t_eval = time.time() - t_eval_start

        train_loss_eval = metrics.get("train")
        val_loss_eval = metrics.get("val")
        train_ppl = ppl_from_loss(train_loss_eval)
        val_ppl = ppl_from_loss(val_loss_eval)

        print(f"[Step {step}] KD iter_loss {iter_train_loss:.4f} | Eval Train {train_loss_eval:.4f} Val {val_loss_eval:.4f} | ValPPL {val_ppl:.2f} | eval_time {t_eval:.1f}s")
        eval_history.append((step, train_loss_eval, val_loss_eval))
        perp_history.append((step, val_ppl))

    if step % SAVE_EVERY == 0 or step == TOTAL_STEPS:
        t_save_start = time.time()
        model_to_save = model.module if isinstance(model, nn.DataParallel) else model
        ckpt_path = CKPT_DIR / f"gpt2_student_step{step}.pt"
        torch.save(model_to_save.state_dict(), ckpt_path)
        tokenizer.save_pretrained(str(TK_DIR))
        save_time_s = time.time() - t_save_start
        print(f"Saved checkpoint step {step} (save_time {save_time_s:.2f}s)")

    # Logging
    with open(log_csv, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        row = [
            step,
            iter_train_loss,
            train_loss_eval if train_loss_eval is not None else "",
            val_loss_eval if val_loss_eval is not None else "",
            ppl_from_loss(train_loss_eval) if train_loss_eval is not None else "",
            val_ppl if val_ppl is not None else "",
            round(step_time, 6),
            save_time_s,
            datetime.utcnow().isoformat()
        ]
        writer.writerow(row)

    timings.append({"step": step, "step_time_s": step_time, "loss": iter_train_loss, "save_time_s": save_time_s})

# End KD training
total_time = time.time() - global_start
print("KD training finished. Total time (s):", total_time)

# Final save
t_final_save_start = time.time()
model_to_save = model.module if isinstance(model, nn.DataParallel) else model
final_path = CKPT_DIR / "gpt2_student_final.pt"
torch.save(model_to_save.state_dict(), final_path)
tokenizer.save_pretrained(str(TK_DIR))
t_final_save = time.time() - t_final_save_start
print("Final student model saved (s):", t_final_save)

# -------------------- INFERENCE TIMING --------------------
print("Generating sample for prompt (timing)...")
t_inf_start = time.time()
generated = generate_text(INFERENCE_PROMPT, max_new_tokens=GENERATE_TOKENS)
t_inf = time.time() - t_inf_start
print("Inference time (s) for {} tokens: {:.3f}s".format(GENERATE_TOKENS, t_inf))

with open(OUT_DIR / "sample_generation.txt", "w", encoding="utf-8") as f:
    f.write("PROMPT:\n" + INFERENCE_PROMPT + "\n\n")
    f.write("GENERATED:\n" + generated + "\n\n")
    f.write(f"Inference time (s): {t_inf}\n")

# -------------------- SUMMARY --------------------
summary = {
    "total_steps": TOTAL_STEPS,
    "batch_size": BATCH_SIZE,

    "student_block_size": STUDENT_BLOCK_SIZE,
    "student_d_model": STUDENT_D_MODEL,
    "student_n_head": STUDENT_N_HEAD,
    "student_n_layer": STUDENT_N_LAYER,
    "student_vocab": STUDENT_VOCAB_SIZE,

    "teacher_block_size": TEACHER_BLOCK_SIZE,
    "teacher_d_model": TEACHER_D_MODEL,
    "teacher_n_head": TEACHER_N_HEAD,
    "teacher_n_layer": TEACHER_N_LAYER,
    "teacher_vocab": TEACHER_VOCAB_SIZE,

    "kd_temperature": TEMPERATURE,
    "kd_alpha": ALPHA,

    "student_parameters": student_params,
    "teacher_parameters": teacher_params,
    "compression_ratio": compression_ratio,

    "final_checkpoint": str(final_path),
    "teacher_checkpoint_used": teacher_ckpt_path,
    "tokenizer_dir": str(TK_DIR),
    "loss_plot": str(OUT_DIR / "loss_curve.png"),
    "perplexity_plot": str(OUT_DIR / "perplexity_curve.png"),
    "timings_csv": str(OUT_DIR / "timings.csv"),
    "inference_time_s": t_inf
}

print("SUMMARY REPORT")
for key, value in summary.items():
    print(f"  {key}: {value}")

print("All outputs saved under:", OUT_DIR)


Starting KD training for 50000 steps ...


/tmp/ipykernel_47/1600034413.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Initial eval - Train 3.9959, Val 3.9951


/tmp/ipykernel_47/1600034413.py:81: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


[Step 2000] KD iter_loss 3.7527 | Eval Train 3.9459 Val 3.9651 | ValPPL 52.73 | eval_time 3.1s
Saved checkpoint step 2000 (save_time 0.15s)
[Step 4000] KD iter_loss 3.5527 | Eval Train 3.8347 Val 3.8370 | ValPPL 46.39 | eval_time 3.1s
Saved checkpoint step 4000 (save_time 0.16s)
[Step 6000] KD iter_loss 3.2876 | Eval Train 3.7038 Val 3.6842 | ValPPL 39.81 | eval_time 3.1s
Saved checkpoint step 6000 (save_time 0.14s)
[Step 8000] KD iter_loss 3.1943 | Eval Train 3.5837 Val 3.5872 | ValPPL 36.13 | eval_time 3.1s
Saved checkpoint step 8000 (save_time 0.16s)
[Step 10000] KD iter_loss 3.0537 | Eval Train 3.4676 Val 3.4449 | ValPPL 31.34 | eval_time 3.1s
Saved checkpoint step 10000 (save_time 0.16s)
[Step 12000] KD iter_loss 2.9655 | Eval Train 3.3338 Val 3.3362 | ValPPL 28.11 | eval_time 3.1s
Saved checkpoint step 12000 (save_time 0.14s)
[Step 14000] KD iter_loss 2.9344 | Eval Train 3.1590 Val 3.1677 | ValPPL 23.75 | eval_time 3.1s
Saved checkpoint step 14000 (save_time 0.14s)
[Step 16000] K